In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.inspection import permutation_importance

%config IPCompleter.greedy = True

In [69]:
ruta_proyecto = r'C:\Users\manug\Desktop\Data Science\DataSets\Wall-Mart\wallmart\\'

In [70]:
nombre_df = 'df_tablon.pickle'

In [71]:
df = pd.read_pickle(ruta_proyecto + '02_Datos/03_Trabajo/' + nombre_df)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357384 entries, 0 to 357383
Data columns (total 69 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   isholiday_False         357384 non-null  float64       
 1   isholiday_True          357384 non-null  float64       
 2   type_A                  357384 non-null  float64       
 3   type_B                  357384 non-null  float64       
 4   type_C                  357384 non-null  float64       
 5   isholiday_lag_1_False   357384 non-null  float64       
 6   isholiday_lag_1_True    357384 non-null  float64       
 7   isholiday_lag_2_False   357384 non-null  float64       
 8   isholiday_lag_2_True    357384 non-null  float64       
 9   isholiday_lag_3_False   357384 non-null  float64       
 10  isholiday_lag_3_True    357384 non-null  float64       
 11  isholiday_lag_4_False   357384 non-null  float64       
 12  isholiday_lag_4_True    357384

### METODOS SUPERVISADOS

#### Preparación x e y

In [73]:
target = 'weekly_sales'

In [77]:
x = df.drop(columns = ['date', 'store', 'weekly_sales']).copy()
y = df[target].copy()

#### Mutual Information

In [78]:
mutual_selector = mutual_info_regression(x, y)

In [79]:
mutual_selector

array([1.18336309e-03, 1.10825650e-03, 3.09003628e-02, 2.09911857e-02,
       4.48837417e-02, 4.54128645e-04, 3.19444174e-04, 3.43712921e-04,
       8.71999146e-04, 8.69830067e-04, 9.41391312e-04, 7.69342240e-04,
       8.15283578e-04, 1.27626244e-03, 6.50202939e-02, 1.06076329e-03,
       9.87413231e-04, 7.32079646e-04, 8.20218895e-04, 2.07499165e-02,
       1.56703970e-02, 1.71658783e-02, 4.59043054e-03, 8.71733524e-03,
       6.84303623e-03, 1.66462486e-02, 2.86236528e-02, 9.24244836e-02,
       1.52852785e-01, 8.42363242e-01, 2.01651242e+00, 2.04953756e-02,
       1.53121407e-02, 2.91686869e-02, 9.22908373e-02, 1.82384144e+00,
       2.05604935e-02, 1.55269426e-02, 2.88720630e-02, 9.28391117e-02,
       1.72279946e+00, 2.10572801e-02, 1.56933244e-02, 2.87869641e-02,
       9.24671079e-02, 1.69482312e+00, 2.10148437e-02, 1.60467342e-02,
       2.90383260e-02, 9.26171355e-02, 2.71640161e+00, 2.09648430e-02,
       1.75433584e-02, 3.00221790e-02, 9.27264661e-02, 2.24695915e+00,
      

In [80]:
def ranking_mi(mutual_selector, modo = 'tabla'):
    #Maqueta el ranking
    ranking_mi = pd.DataFrame(mutual_selector, index = x.columns).reset_index()
    ranking_mi.columns = ['variable','importancia_mi']
    ranking_mi = ranking_mi.sort_values(by = 'importancia_mi', ascending = False)
    ranking_mi['ranking_mi'] = np.arange(0,ranking_mi.shape[0])
    #Muestra la salida
    if modo == 'tabla':
        return(ranking_mi)
    else:
        g = ranking_mi[0:45].importancia_mi.sort_values().plot.barh(figsize = (12,8))
        g.set_yticklabels(ranking_mi[0:45].sort_values(by = 'importancia_mi').variable)
        plt.show()
        return(g)

In [81]:
ranking_mi = ranking_mi(mutual_selector, modo = 'tabla');

In [82]:
ranking_mi.tail(50)

,variable,importancia_mi,ranking_mi
34,unemployment_lag_1_mms,0.092291,15
64,unemployment_mm_8_mms,0.091888,16
14,type_te,0.065020,17
4,type_C,0.044884,18
58,cpi_mm_4_mms,0.030973,19
2,type_A,0.030900,20
63,cpi_mm_8_mms,0.030243,21
53,cpi_mm_2_mms,0.030022,22
33,cpi_lag_1_mms,0.029169,23
48,cpi_lag_4_mms,0.029038,24


#### Seleccionar las variables que pasan

In [88]:
posicion_variable_limite = 46

In [89]:
entran_mi = ranking_mi.iloc[0:posicion_variable_limite].variable

In [90]:
x_mi = x[entran_mi].copy()
x_mi

,weekly_sales_mm_2_mms,weekly_sales_mm_4_mms,weekly_sales_lag_1_mms,weekly_sales_mm_8_mms,weekly_sales_lag_2_mms,weekly_sales_lag_3_mms,weekly_sales_lag_4_mms,dept_mms,size_mms,unemployment_lag_2_mms,...,temperature_lag_2_mms,temperature_lag_1_mms,fuel_price_mm_4_mms,fuel_price_mm_2_mms,markdown1_mms,markdown5_mms,fuel_price_lag_4_mms,fuel_price_lag_3_mms,fuel_price_mms,fuel_price_lag_2_mms
0,0.057131,0.069901,0.038857,0.102919,0.037291,0.038415,0.034942,0.000000,0.630267,0.393611,...,0.554207,0.554207,0.116992,0.118663,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
1,0.097696,0.133360,0.069625,0.167232,0.069821,0.076476,0.070595,0.010204,0.630267,0.393611,...,0.554207,0.523581,0.118663,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
2,0.025460,0.033576,0.020041,0.042149,0.021648,0.024731,0.023098,0.020408,0.630267,0.393611,...,0.523581,0.523581,0.120334,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.122006
3,0.078564,0.104786,0.056020,0.131159,0.053944,0.061704,0.056797,0.030612,0.630267,0.393611,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.115320,0.122006,0.122006,0.122006
4,0.071589,0.080878,0.039567,0.100838,0.037943,0.040211,0.041672,0.040816,0.630267,0.393611,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.122006,0.122006,0.122006,0.122006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357379,0.012933,0.016792,0.013349,0.018471,0.016531,0.013535,0.012940,0.938776,0.451136,0.450078,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357380,0.010426,0.012421,0.009989,0.013434,0.012398,0.012430,0.010919,0.948980,0.451136,0.450078,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357381,0.123414,0.171109,0.087250,0.209947,0.096830,0.089377,0.088261,0.959184,0.451136,0.450078,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357382,0.019104,0.023781,0.016667,0.027779,0.018808,0.016799,0.016987,0.979592,0.451136,0.450078,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187


#### Se evaluaron los sigueintes metodos de preselaccion de variable:
* Mutual Information
* Recursive Feature Elimination
* Permutation Importance

Se seleccionó el metodo Mutual Information para la preseleccion de variables por lo que fueron los resultados arrojados de cada metodo

#### Seleccionar método final

In [91]:
x_preseleccionada = x_mi

#### Incorporar variables de segmentación

In [92]:
x_preseleccionada = pd.concat([df[['date', 'store', 'weekly_sales']], x_preseleccionada], axis = 1)

In [93]:
x_preseleccionada

,date,date,store,weekly_sales,weekly_sales_mm_2_mms,weekly_sales_mm_4_mms,weekly_sales_lag_1_mms,weekly_sales_mm_8_mms,weekly_sales_lag_2_mms,weekly_sales_lag_3_mms,...,temperature_lag_2_mms,temperature_lag_1_mms,fuel_price_mm_4_mms,fuel_price_mm_2_mms,markdown1_mms,markdown5_mms,fuel_price_lag_4_mms,fuel_price_lag_3_mms,fuel_price_mms,fuel_price_lag_2_mms
0,2010-03-26,2010-03-26,1,26229.21,0.057131,0.069901,0.038857,0.102919,0.037291,0.038415,...,0.554207,0.554207,0.116992,0.118663,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
1,2010-03-26,2010-03-26,1,41892.55,0.097696,0.133360,0.069625,0.167232,0.069821,0.076476,...,0.554207,0.523581,0.118663,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
2,2010-03-26,2010-03-26,1,10366.85,0.025460,0.033576,0.020041,0.042149,0.021648,0.024731,...,0.523581,0.523581,0.120334,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.122006
3,2010-03-26,2010-03-26,1,33872.07,0.078564,0.104786,0.056020,0.131159,0.053944,0.061704,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.115320,0.122006,0.122006,0.122006
4,2010-03-26,2010-03-26,1,38971.89,0.071589,0.080878,0.039567,0.100838,0.037943,0.040211,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.122006,0.122006,0.122006,0.122006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357379,2012-07-20,2012-07-20,45,3568.64,0.012933,0.016792,0.013349,0.018471,0.016531,0.013535,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357380,2012-07-20,2012-07-20,45,3618.52,0.010426,0.012421,0.009989,0.013434,0.012398,0.012430,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357381,2012-07-20,2012-07-20,45,53136.34,0.123414,0.171109,0.087250,0.209947,0.096830,0.089377,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357382,2012-07-20,2012-07-20,45,6902.68,0.019104,0.023781,0.016667,0.027779,0.018808,0.016799,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187


#### Guardar datasets

In [94]:
nombre_x_preseleccionado = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'x_preseleccionado.pickle'
nombre_y_preseleccionado = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'y_preseleccionado.pickle'

In [95]:
x_preseleccionada.to_pickle(nombre_x_preseleccionado)

y_preseleccionado = y.copy()
y_preseleccionado.to_pickle(nombre_y_preseleccionado)